# 使用量子生成對抗網路為期權定價

## 簡介

#### 在此筆記本，我們討論量子機器學習，即量子生成對抗網路(qGAN)，如何促進歐洲看漲期權的定價。更具體的說，qGAN可以訓練成量子電路模擬歐洲看漲期權基礎資產的現貨價格。由此得到的模型可以集成到基於量子振幅估計的算法中，以評估預期的收益-參見歐洲看漲期權定價。關於通過訓練qGAN學習和加載隨機分佈的更多細節，請參閱論文''Quantum Generative Adversarial Networks for Learning and Loading Random Distributions. Zoufal, Lucchi, Woerner. 2019.''

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from qiskit import Aer, QuantumRegister, QuantumCircuit, BasicAer
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import TwoLocal, NormalDistribution
from qiskit.quantum_info import Statevector

from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import IterativeAmplitudeEstimation
from qiskit.finance.applications import EuropeanCallExpectedValue

D:\Anaconda\lib\site-packages\qiskit\aqua\__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
D:\Anaconda\lib\site-packages\qiskit\finance\__init__.py:50: DeprecationWarning: The package qiskit.finance is deprecated. It was moved/refactored to qiskit_finance (pip install qiskit-finance). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('finance', 'qiskit_finance', 'qiskit-finance')


##  不确定性模型 Uncertainty Model
#### Black-Scholes模型假設歐洲看漲期權在到期時 $S_T$ 的現貨價格是對數正態分佈的。因此，我們可以用來自對數正態分佈的樣本訓練qGAN，並將結果作為基於期權的不確定性模型。接下来，我們構建一個量子電路用於加載不確定性模型。電路輸出如下

<center>$ {∣g_\theta⟩} = \sum_{j=0}^{2^n−1}\sqrt{p_θ^j}{∣j⟩}$</center>

#### 其中概率 $p_θ^j$ ($j∈{0，…，2n−1}$)，表示目標分佈模型。

In [2]:
# 设置上、下数据值
bounds = np.array([0.,7.])
# 不确定性模型中使用的量子位元的集合数
num_qubits = 6

#加载训练过的电路参数
# g_params = [0.29399714, 0.38853322, 0.9557694, 0.07245791, 6.02626428, 0.13537225]
g_params = [0.2939, 0.3885, 0.9557, 0.0724, 6.0262, 0.1353]

# 设置发电机电路的初始状态
init_dist = NormalDistribution(num_qubits, mu=1., sigma=1., bounds=bounds)
print(init_dist)

# 构造变分形式
var_form = TwoLocal(num_qubits, 'ry', 'cz', entanglement='circular', reps=1, insert_barriers=True)
print(var_form)
# 保留一个参数列表，以便我们可以将它们与数值列表关联起来
# (否则我们需要一个字典)
theta = var_form.ordered_parameters

# 组成生成电路，这是加载不确定性模型的电路
g_circuit = init_dist.compose(var_form)
# print(g_circuit)

<ipython-input-2-d27b115ac203>:11: DeprecationWarning: `NormalDistribution` is deprecated as of version 0.17.0 and will be removed no earlier than 3 months after the release date. It moved to qiskit_finance.circuit.library.NormalDistribution.
  init_dist = NormalDistribution(num_qubits, mu=1., sigma=1., bounds=bounds)


     ┌───────┐
q_0: ┤0      ├
     │       │
q_1: ┤1      ├
     │       │
q_2: ┤2      ├
     │  P(X) │
q_3: ┤3      ├
     │       │
q_4: ┤4      ├
     │       │
q_5: ┤5      ├
     └───────┘
     ┌──────────┐ ░                    ░  ┌──────────┐
q_0: ┤ Ry(θ[0]) ├─░──■──■──────────────░──┤ Ry(θ[6]) ├
     ├──────────┤ ░  │  │              ░  ├──────────┤
q_1: ┤ Ry(θ[1]) ├─░──┼──■──■───────────░──┤ Ry(θ[7]) ├
     ├──────────┤ ░  │     │           ░  ├──────────┤
q_2: ┤ Ry(θ[2]) ├─░──┼─────■──■────────░──┤ Ry(θ[8]) ├
     ├──────────┤ ░  │        │        ░  ├──────────┤
q_3: ┤ Ry(θ[3]) ├─░──┼────────■──■─────░──┤ Ry(θ[9]) ├
     ├──────────┤ ░  │           │     ░ ┌┴──────────┤
q_4: ┤ Ry(θ[4]) ├─░──┼───────────■──■──░─┤ Ry(θ[10]) ├
     ├──────────┤ ░  │              │  ░ ├───────────┤
q_5: ┤ Ry(θ[5]) ├─░──■──────────────■──░─┤ Ry(θ[11]) ├
     └──────────┘ ░                    ░ └───────────┘
     ┌───────┐»
q_0: ┤0      ├»
     │       │»
q_1: ┤1      ├»
     │       │»
q_2: ┤2   

## 評估預期收益

#### 現在，訓練的不確定性模型可以用量子振幅估計來評估期權收益函數的期望值。

In [3]:
#设置执行价格(应该在不确定性的低值和高值范围内)
strike_price = 2

#设置成本函数的近似缩放
c_approx = 0.25

#为成本函数构造电路
european_call_objective = EuropeanCallExpectedValue(
    num_qubits,
    strike_price=strike_price,
    rescaling_factor=c_approx,
    bounds=bounds
)
european_call = european_call_objective.compose(g_circuit, front=True)
print(european_call)

        ┌───────┐»
q126_0: ┤0      ├»
        │       │»
q126_1: ┤1      ├»
        │       │»
q126_2: ┤2      ├»
        │  P(X) │»
q126_3: ┤3      ├»
        │       │»
q126_4: ┤4      ├»
        │       │»
q126_5: ┤5      ├»
        └───────┘»
q127_0: ─────────»
                 »
  a0_0: ─────────»
                 »
  a0_1: ─────────»
                 »
  a0_2: ─────────»
                 »
  a0_3: ─────────»
                 »
  a0_4: ─────────»
                 »
  a0_5: ─────────»
                 »
«        ┌──────────────────────────────────────────────────────────────────────────┐»
«q126_0: ┤0                                                                         ├»
«        │                                                                          │»
«q126_1: ┤1                                                                         ├»
«        │                                                                          │»
«q126_2: ┤2                                         

## 繪製概率分佈

##### 接下來，我們繪製訓練後的概率分佈圖，為了便於比較，我們也繪製目標概率分佈圖。

In [4]:
# 评估训练过的概率分布
values = [bounds[0] + (bounds[1] - bounds[0]) * x / (2 ** num_qubits - 1) for x in range(2**num_qubits)]
uncertainty_model = g_circuit.assign_parameters(dict(zip(theta, g_params)))
amplitudes = Statevector.from_instruction(uncertainty_model).data

x = np.array(values)
y = np.abs(amplitudes) ** 2

# 从目标概率分布中抽取样本
N = 100000
log_normal = np.random.lognormal(mean=1, sigma=1, size=N)
log_normal = np.round(log_normal)
log_normal = log_normal[log_normal <= 7]
log_normal_samples = []
for i in range(8):
    log_normal_samples += [np.sum(log_normal==i)]
log_normal_samples = np.array(log_normal_samples / sum(log_normal_samples))

# 画出分布
plt.bar(x, y, width=0.2, label='trained distribution', color='royalblue')
plt.xticks(x, size=15, rotation=90)
plt.yticks(size=15)
plt.grid()
plt.xlabel('Spot Price at Maturity $S_T$ (\$)', size=15)
plt.ylabel('Probability ($\%$)', size=15)
plt.plot(log_normal_samples,'-o', color ='deepskyblue', label='target distribution', linewidth=4, markersize=12)
plt.legend(loc='best')
plt.show()

TypeError: ParameterExpression with unbound parameters ({ParameterVectorElement(θ[6])}) cannot be cast to a float.

## 評估預期收益

#### 現在，訓練的不確定性模型可以用量子振幅估計的方法解析求解期權收益函數的期望值。

In [ ]:
#评估不同发行版本的收益
payoff = np.array([0,0,0,1,2,3,4,5])
ep = np.dot(log_normal_samples, payoff)
print("Analytically calculated expected payoff w.r.t. the target distribution:  %.4f" % ep)
ep_trained = np.dot(y, payoff)
print("Analytically calculated expected payoff w.r.t. the trained distribution: %.4f" % ep_trained)

#绘制精确的收益函数(在训练的不确定性模型的网格上进行评估)
x = np.array(values)
y_strike = np.maximum(0, x - strike_price)
plt.plot(x, y_strike, 'ro-')
plt.grid()
plt.title('Payoff Function', size=15)
plt.xlabel('Spot Price', size=15)
plt.ylabel('Payoff', size=15)
plt.xticks(x, size=15, rotation=90)
plt.yticks(size=15)
plt.show()

In [ ]:
#为QAE构造一个操作符
european_call = european_call_objective.compose(uncertainty_model, front=True)
print(european_call)

In [ ]:
#设置目标精度和置信水平
epsilon = 0.01
alpha = 0.05

# 构造振幅估计
ae = IterativeAmplitudeEstimation(epsilon=epsilon, alpha=alpha,
                                  state_preparation=european_call,
                                  objective_qubits=[num_qubits],
                                  post_processing=european_call_objective.post_processing)
print(ae)

In [ ]:
result = ae.run(quantum_instance=Aer.get_backend('qasm_simulator'), shots=1000)

In [ ]:
conf_int = np.array(result['confidence_interval'])
print('Exact value:        \t%.4f' % ep_trained)
print('Estimated value:    \t%.4f' % (result['estimation']))
print('Confidence interval:\t[%.4f, %.4f]' % tuple(conf_int))